In [6]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [9]:
df = pd.read_csv('mutations.csv',encoding = "ISO-8859-1", error_bad_lines=False, names = ["Class", "Package", "gc1", "gc2","gc3","Coverage","gc4"])

In [10]:
df.drop('gc1', axis=1, inplace=True)
df.drop('gc2', axis=1, inplace=True)
df.drop('gc3', axis=1, inplace=True)
df.drop('gc4', axis=1, inplace=True)
df['Package'] = df['Package'].map(lambda x: str(x)[:x.rfind('.')])
df['Package'] = df['Package'].map(lambda x:  x if (x.find('$')+1 == 0) else  x[:x.find('$')+1] )
df['Class'] = df['Class'].map(lambda x: str(x)[:x.rfind('.java')])
df.rename(columns={'Class':'CLASS',
                          'Package':'PACKAGE'},
                 inplace=True)
df.head()


,CLASS,PACKAGE,Coverage
0,Clas,Packag,Coverage
1,DoubleMetaphone,org.apache.commons.codec.language,KILLED
2,DoubleMetaphone,org.apache.commons.codec.language,KILLED
3,DoubleMetaphone,org.apache.commons.codec.language,KILLED
4,DoubleMetaphone,org.apache.commons.codec.language,KILLED


In [11]:
df = df.groupby(['CLASS','PACKAGE','Coverage'],as_index = False).size().unstack(fill_value=0)


In [12]:
df['Total_Mutant'] =  (df['KILLED'] + df['NO_COVERAGE'] + df['SURVIVED'] + df['TIMED_OUT'])
df['Mutation_Score'] =  ((df['KILLED']+df['TIMED_OUT']) / df['Total_Mutant'])*100


In [14]:
df1 = pd.read_csv('jacoco.csv', error_bad_lines=False)
df1['CLASS'] = df1['CLASS'].map(lambda x:  x if (x.find('.')+1 == 0) else  x[:x.find('.')] )
df1 = df1.groupby(df1['CLASS']).aggregate(sum).reset_index()
df1.columns

Index(['CLASS', 'INSTRUCTION_MISSED', 'INSTRUCTION_COVERED', 'BRANCH_MISSED',
       'BRANCH_COVERED', 'LINE_MISSED', 'LINE_COVERED', 'COMPLEXITY_MISSED',
       'COMPLEXITY_COVERED', 'METHOD_MISSED', 'METHOD_COVERED',
       'Statement Coverage', 'Branch Coverage', 'Code Complexity'],
      dtype='object')

In [15]:
merged_inner = pd.merge(left=df,right=df1, left_on='CLASS', right_on='CLASS')


In [16]:
df1[(~df1.CLASS.isin(merged_inner.CLASS))&(~df1.CLASS.isin(merged_inner.CLASS))]


,CLASS,INSTRUCTION_MISSED,INSTRUCTION_COVERED,BRANCH_MISSED,BRANCH_COVERED,LINE_MISSED,LINE_COVERED,COMPLEXITY_MISSED,COMPLEXITY_COVERED,METHOD_MISSED,METHOD_COVERED,Statement Coverage,Branch Coverage,Code Complexity
4,Base32InputStream,0,24,0,0,0,6,0,3,0,3,100.0,0.0,3
5,Base32OutputStream,0,24,0,0,0,6,0,3,0,3,100.0,0.0,3
7,Base64InputStream,0,24,0,0,0,6,0,3,0,3,100.0,0.0,3
8,Base64OutputStream,0,24,0,0,0,6,0,3,0,3,100.0,0.0,3
17,CharEncoding,0,3,0,0,0,1,0,1,0,1,100.0,0.0,1
23,DecoderException,0,16,0,0,0,8,0,4,0,4,100.0,0.0,4
27,EncoderException,0,16,0,0,0,8,0,4,0,4,100.0,0.0,4
47,ResourceConstants,3,0,0,0,1,0,1,0,1,0,0.0,0.0,1


In [17]:
df = merged_inner
df.columns
merged_inner.head()


,CLASS,Coverage,KILLED,NON_VIABLE,NO_COVERAGE,SURVIVED,TIMED_OUT,Total_Mutant,Mutation_Score,INSTRUCTION_MISSED,INSTRUCTION_COVERED,BRANCH_MISSED,BRANCH_COVERED,LINE_MISSED,LINE_COVERED,COMPLEXITY_MISSED,COMPLEXITY_COVERED,METHOD_MISSED,METHOD_COVERED,Statement Coverage,Branch Coverage,Code Complexity
0,AbstractCaverphone,0,2,0,0,1,0,3,66.666667,0,24,0,2,0,6,0,4,0,3,100.000000,100.000000,4
1,B64,0,16,0,0,0,1,17,100.000000,0,72,0,4,0,13,0,7,0,5,100.000000,100.000000,7
2,BCodec,0,25,0,3,1,0,29,86.206897,27,119,2,18,7,37,3,21,1,13,84.090909,90.000000,24
3,Base32,0,201,0,10,14,1,226,89.380531,153,2550,13,64,17,157,14,42,1,12,90.229885,83.116883,56
4,Base64,0,154,0,1,18,1,174,89.080460,30,1936,7,97,4,149,8,74,1,27,97.385621,93.269231,82


In [18]:
df.plot(x='Mutation_Score', y='Statement Coverage', style='o')

df[['Mutation_Score','Statement Coverage']].corr(method ='spearman')

df.plot(x='Mutation_Score', y='Branch Coverage', style='o')

df[['Mutation_Score','Branch Coverage']].corr(method ='spearman')


,Mutation_Score,Branch Coverage
Mutation_Score,1.000000,0.213251
Branch Coverage,0.213251,1.000000
